In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

In [2]:
import os
from glob import glob
import random
import time
import datetime

from tqdm import tqdm

import numpy as np
import pandas as pd
from IPython.display import FileLink
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import cv2

from sklearn.utils import shuffle
from sklearn.metrics import log_loss
from scipy import spatial

import re

from sklearn.ensemble import GradientBoostingClassifier
from feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats
from feature_engineering import word_overlap_features
from utils.dataset import DataSet
from utils.generate_test_splits import kfold_split, get_stances_for_folds
from utils.score import report_score, LABELS, score_submission


from utils.system import parse_params, check_version

#our additions
import json
import sklearn

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.hidden = nn.Linear(75, 50)
        self.hidden2 = nn.Linear(50, 25)
        self.output = nn.Linear(25, 10)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.hidden2(x)
        x = self.sigmoid(x)
        x = self.output(x)
        return x

In [4]:
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=.5)

In [ ]:
# Need to add the Part where i create the Training Data (working on that with the embedding now.)

In [5]:
#test variable batch size
batch_size = 50
epochs = 5
track_loss = []
for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    if epoch > 0 and epoch%40==0:
        if batch_size < 800:
            batch_size *= 2
        print("batch size:", batch_size)
    for i in range(0, len(X_train), batch_size):
        # get the inputs; data is a list of [inputs, labels]
        print(i)
        poses = [pose[0] for pose in X_train[i:i+batch_size]]
        imgs   = [images[1] for images in X_train[i:i+batch_size]]
        inputs = torch.FloatTensor(poses)
        labels = torch.LongTensor(Y_train[i:i+batch_size])
        img_t = torch.FloatTensor(imgs)
        cnn_outputs = model(img_t)
        # print(labels)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        # print(out.size())
        outputs = net(inputs, cnn_outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    track_loss.append(running_loss * batch_size / 1000)
    print('[Epoch: %d] loss: %.3f' % (epoch + 1, running_loss * batch_size / 1000))
    

print('Finished Training')

NameError: name 'X_train' is not defined